In [112]:
# Creating a function that will create the goal state based
# on the number of dimensions that the user gives it.
# Arguments:
# N: The number of rows and columns for the square matrix.
def puzzle(N):
    # Creating an matrix of dimension NxN with all 0's
    mat = [N*[0] for i in range(N)]
    # Iterating over all rows
    for r in range(N):
        # For each row, iterating over each column
        for c in range(N):
            #print(r,c)
            # Assigning the corresponding value of the puzzle
            # by using the following formula.
            mat[r][c] = N*r + c%N + 1
            
    # Based on the formula used for each element above, the [N,N] position
    # will contain (N+1) but we want it to be 0 to represent an empty space
    mat[N-1][N-1] = 0
    
    return mat

In [113]:
puzzle(2)

[[1, 2], [3, 0]]

In [115]:
# This function creates a dictionary with the correct indices for each of 
# the tiles in the given state.
def mat2dict(mat):
    # Defining an intial empty dictiory
    goal_dict = {}
    # The dimension of mat
    n = len(mat)
    # Iterating over each element in the matrix
    for i in range(n):
        for j in range(n):
            # The value of the i,j tile
            tile = mat[i][j]
            #print(tile)
            # For each tile, assign its corresponding indices
            goal_dict[tile] = [i,j]
    
    return goal_dict

In [116]:
# Testing the function mat2dict
mat2dict(puzzle(2))

{0: [1, 1], 1: [0, 0], 2: [0, 1], 3: [1, 0]}

In [66]:
10//3

3

In [ ]:
# Creating a function for the A* algorithm that will work for 
# uniform cost search, the misplaced tile heuristic and the 
# manhatten distance heuristic.
# The arguments for the function are:
# goal_mat: The goal state for the puzzle
# start_mat: The puzzle that we are starting the search with.
# g: A cost function
# h: A heuristic function
def astar(goal_mat, start_mat, g, h):
    # defining the cost that w

In [ ]:
def h_0(crnt, goal):
    return 0

In [117]:
def h_misplace(crnt, goal):
    # number of rows and columns in crnt
    n = len(crnt)
    # Counter for the number of misplaced tiles
    misplace = 0
    # iterating over all elements in the crnt and goal matrices
    for i in range(n):
        for j in range(n):
            print(crnt[i][j], goal[i][j])
            # Checking to see if the i,j element in crnt state 
            # is in the right place or not. Including a condition
            # that the misplaced element can't equal 0 since this
            # heuristic doesn't consider it as misplaced.
            if crnt[i][j] != goal[i][j] and crnt[i][j] != 0:
                # Each time we have a misplaced tile, misplace increases by 1
                misplace += 1
    return misplace

In [141]:
state = [[1,2,4],[3,0,6],[7,8,5]]
h_misplace(state, puzzle(3))

1 1
2 2
4 3
3 4
0 5
6 6
7 7
8 8
5 0


3

In [152]:
def h_manhatt(crnt, goal):
    # Dimension of crnt and goal
    n = len(crnt)
    # Creating a variable for the manhattan distance
    manhatt_dist = 0
    # Iterating over all elements in the matrix
    for i in range(n):
        for j in range(n):
            # Don't want to calculate manhattan distance for tile 0
            if crnt[i][j] != 0:
                # The value of the i,j tile in crnt
                tile = crnt[i][j]
                # The correct indices based on the goal state
                # for the value tile
                crct_ind = mat2dict(goal)[tile]
                # Calculating the manhattan distance
                manhatt_dist += abs(i - crct_ind[0]) + abs(j - crct_ind[1])
    return manhatt_dist

In [153]:
# Testing h_manhatt
state = [[1,2,3],[5,4,6],[7,8,0]]
h_manhatt(state, puzzle(3))
#state

8

In [ ]:
# Function that determines whether or not the search has 
# reached the goal state given the current state.
# Arguments:
# crnt: The current state in the search.
# goal: the goal state of the search.
# Return:
# 
def goal_state(crnt, goal):
    # Compares the crnt and goal matrices to determine if they 
    # are equivalent.
    return(crnt == goal)

In [27]:
mat = [[1,0],[0,1]]
mat[0]

[1, 0]

In [135]:
abs(44)

44

In [31]:
N=3
mat = [3*[0] for i in range(N)]
mat

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

In [34]:
mat[0][0] = 1
mat[2][2] = 4
mat

[[1, 0, 0], [0, 0, 0], [0, 0, 4]]

In [32]:
mat1 = [3*[0] for i in range(N)]
mat1

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

In [35]:
mat1 == mat

False

In [23]:
mat[0][0] = 12

In [25]:
mat1

[[12, 0, 0], [0, 0, 0], [0, 0, 4]]

In [67]:
cars = ['kia', 'audi', 'bmw']
for car in enumerate(cars):
  print(car)

(0, 'kia')
(1, 'audi')
(2, 'bmw')


In [73]:
dic = {1:[0,0], 2:[0,1]}

In [97]:
dic[1] = 1

In [98]:
dic

{1: 1, 2: [0, 1]}